In [ ]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

from data import result_loader, plotter
from data.plotter import PlotType as PT

# Experiment evaluation
This interactive notebook loads specs, histories and models from files generated by a past experiment and plots the results.

The next cell prints all specs-files from the subdirectory `/results` and asks for the experiment to load.
Simply copy a complete line into the text field and press enter to perform the evaluation.

In [ ]:
relative_paths = result_loader.get_relative_spec_file_paths()
sep = '\n'
message = f"{sep.join(relative_paths)}\n\nPlease choose a specs-file: "
specs_file = input(message)
experiment_prefix = result_loader.extract_experiment_path_prefix(specs_file)
absolute_specs_path = result_loader.generate_absolute_specs_path(specs_file)

In [ ]:
# load experiment's specs and results
specs = result_loader.get_specs_from_file(absolute_specs_path, as_dict=False)
hists = result_loader.get_histories_from_file(experiment_prefix)
nets = result_loader.get_models_from_files(experiment_prefix, specs)

In [ ]:
specs

In [ ]:
plotter.plot_two_average_hists(hists.val_acc, hists.train_loss, hists.sparsity, specs.plot_step, PT.VAL_ACC, PT.TRAIN_LOSS, force_zero_left=False, force_zero_right=True)

In [ ]:
plotter.plot_two_average_hists(hists.train_loss, hists.val_loss, hists.sparsity, specs.plot_step, PT.TRAIN_LOSS, PT.VAL_LOSS, force_zero_left=True, force_zero_right=True)

In [ ]:
plotter.plot_acc_at_early_stopping(hists.val_acc, hists.val_loss, hists.sparsity, PT.VAL_ACC)

In [ ]:
plotter.plot_early_stop_iterations(hists.val_loss, hists.sparsity, specs.plot_step)

In [ ]:
plotter.plot_average_hists(hists.test_acc, hists.sparsity, specs.plot_step, PT.TEST_ACC)

In [ ]:
# compare final test-accuracies
for n in range(specs.net_count):
    unpruned_final = hists.test_acc[n, 0, -1]
    for p in range(1, specs.prune_count + 1):
        pruned_final = hists.test_acc[n, p, -1]
        print(f"Test-accuracy net #{n} (sparsity: {hists.sparsity[p]:.4}): {pruned_final:1.4} (changed by {(pruned_final-unpruned_final):1.4})")

In [ ]:
nets[0].sparsity_report()